Ссылка на colab
https://colab.research.google.com/drive/1CTtzlNwkihbcXFhVVFkSt9hMEbQemoh7?usp=sharing

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
!pip install transformers

In [ ]:
!pip install catboost

### Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
import warnings
import lightgbm as lgb
import time

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

from tqdm import notebook
from tqdm import tqdm

from catboost import CatBoostClassifier, Pool

In [ ]:
warnings.filterwarnings('ignore')

### Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 df = pd.read_csv('/content/drive/MyDrive/01 DS/01 Yandex Practicum/toxic_comments/toxic_comments.csv')

## Обучение

Загрузка предобученных модели и токенизатора.

In [ ]:
model = transformers.AutoModel.from_pretrained('unitary/toxic-bert')
tokenizer = transformers.AutoTokenizer.from_pretrained('unitary/toxic-bert')

Преобразуем текст

In [ ]:
pre_tokenized = df['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True))

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


Как видим из сообщения к pre_tokenized, имеем ограничение в 512 токенов в строке. Создадим такое ограничение.

Собираем датасет из токенизированных данных - столбец text, целевого признака и длин списков токенов.

In [ ]:
df_temp = pd.DataFrame(pre_tokenized)
df_temp['toxic'] = df['toxic']
len_list = []
for i in df_temp['text']:
    len_list.append(len(i))
df_temp['len'] = len_list
df_temp

,text,toxic,len
0,"[101, 7526, 2339, 1996, 10086, 2015, 2081, 210...",0,68
1,"[101, 1040, 1005, 22091, 2860, 999, 2002, 3503...",0,35
2,"[101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...",0,54
3,"[101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...",0,144
4,"[101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...",0,21
...,...,...,...
159287,"[101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...",0,68
159288,"[101, 2017, 2323, 2022, 14984, 1997, 4426, 200...",0,27
159289,"[101, 13183, 6290, 26114, 1010, 2045, 2015, 20...",0,19
159290,"[101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...",0,28


Удаляем строки с превышающими лимит токенами. таких строк порядка 1.8%

In [ ]:
df_temp = df_temp.loc[df_temp['len'] < 513].drop(['len'], axis=1)

Собираем данные по индексу: слева - тексты из исходного датасета, справа - целевой признак из нового, отфильтрованного датасета. Так как join имеет значение inner, то остаются только те строки, в которых совпадает индекс, то есть прошедшие фильтрацию.

In [ ]:
df_left = df['text']
df_right = df_temp['toxic']
result = pd.concat([df_left, df_right], axis=1, join="inner")
result

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


В случае преобразования в эмбенддинги внутри ноутбука, код ниже - указывает количество семплов для будущего датасета, где q_sample = len(result) - это полный датасет.

Ниже приведен пример расчета для очень маленькой выборки - взяли 256 строк, чтобы показать работоспособность кода.

Полный преобразованный датасет будет использоваться уже на этапе разделения признаков и целевого признака.

In [ ]:
q_sample = 4096 # нужное число для датасета, вплоть до q_sample = len(result)
result = result.sample(q_sample).reset_index(drop=True)

### Преобразуем текст предобработанных данных

Токенизируем отфильтрованную выборку.

In [ ]:
tokenized = result['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True))

Предупреждения больше нет. Максимальная длина токена max_len равна ограничению - 512. Вынесли это значения в константы.

При исполнении преобразования внутри ноутбука, max_len расчитывается ниже. Так как мы случайным образом отберем какое-то количество строк, может случиться, что строки будут короче константной max_len = 512, это нужно учитывать

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

512

### Преобразование векторов

Паддингом добавляем к каждому вектору нули в конец, так, чтобы длина каждого вектора была равна длине max_len.

Маска тоже применяется к каждому вектору - значению, отличному от нуля, присваивается 1, нули остаются нулями.

In [ ]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

### Преобразование в эмбеддинги

In [ ]:
%%time
from tqdm import notebook
batch_size = 64 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings

df_ed = pd.DataFrame(np.concatenate(embeddings)) # соберём все эмбеддинги в матрицу признаков вызовом функции concatenate()
df_ed['toxic'] = result['toxic'] # добавляем целевой признак
df_ed

  0%|          | 0/64 [00:00<?, ?it/s]

CPU times: user 2min 9s, sys: 492 ms, total: 2min 9s
Wall time: 2min 16s


,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,toxic
0,-0.586298,-0.939466,0.659756,-0.421844,0.985815,0.292342,-0.134406,0.034419,-0.389288,-0.620233,...,-1.336425,0.248437,-0.657485,0.085443,0.865121,-0.376805,-0.737543,0.387663,0.142325,0
1,-0.831884,-0.865431,0.473457,-0.384296,1.035798,0.240806,-0.190848,0.060595,-0.324949,-0.617155,...,-1.266620,0.152800,-0.738499,0.103554,0.898390,-0.574140,-0.758753,0.477778,0.122623,0
2,-0.328855,0.504372,0.781910,0.212954,-0.548644,0.684209,1.324862,0.645755,-0.348915,0.475758,...,0.925567,-0.536889,-0.557106,-0.830609,-0.632906,-0.130944,-0.300695,0.176920,0.300887,1
3,-0.598833,-0.917849,0.679589,-0.345724,0.975384,0.124146,-0.138577,0.141820,-0.332999,-0.661736,...,-1.258238,0.195049,-0.706208,0.161176,0.873706,-0.512138,-0.736954,0.407379,0.071533,0
4,-0.510330,-0.780005,0.560046,-0.423635,0.970720,0.230853,-0.078740,0.020176,-0.381334,-0.636837,...,-1.205820,0.254241,-0.646155,0.173264,0.896006,-0.421585,-0.733386,0.393443,0.159418,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,-0.671228,-0.864717,0.382037,-0.555415,0.911503,0.255179,-0.178668,0.013631,-0.461592,-0.662355,...,-1.216205,0.399859,-0.745357,0.130172,0.951795,-0.439261,-0.724997,0.461565,0.005745,0
4092,-0.605584,-1.078846,0.511130,-0.455643,0.991373,0.380742,-0.094920,0.014984,-0.187413,-0.625181,...,-1.319477,0.124943,-0.653278,0.120543,0.794026,-0.340826,-0.752100,0.574000,0.120635,0
4093,-0.716489,-0.938596,0.477114,-0.446761,1.001965,0.252554,-0.185593,-0.004704,-0.453163,-0.710956,...,-1.271600,0.238129,-0.756693,0.170392,0.902686,-0.515896,-0.725224,0.328117,-0.012677,0
4094,-0.575461,-0.986935,0.647507,-0.456455,0.965278,0.291799,-0.067759,-0.056234,-0.373591,-0.715119,...,-1.230942,0.190661,-0.708521,0.165780,0.919065,-0.389543,-0.757118,0.447476,0.037201,0


In [ ]:
df_ed = df_ed.astype('float32') #меняем тип - чтобы при работе с данными требовалось меньше памяти
df_ed['toxic'] = df_ed['toxic'].astype('int16')

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split( #создаем 4 датасета, два признаков (тест+валидация) и два целевых,
    df_ed.drop(columns='toxic'), #для датасетов признаков удаляем целевой
    df_ed['toxic'], #для целевого оставляем только целевой
    test_size=0.2, #с соотношением
    random_state=42, #с заданной опорой для рандома
    stratify= df_ed['toxic']) #с заданной стратификацией по целевому признаку

In [ ]:
X_train, X_val, y_train, y_val = train_test_split( #создаем 4 датасета, два признаков (тест+валидация) и два целевых,
    X_train_val,
    y_train_val,
    test_size=0.2, #с соотношением
    random_state=42, #с заданной опорой для рандома
    stratify=y_train_val) #с заданной стратификацией по целевому признаку

### CatBoostClassifier

In [ ]:
start_time = time.time()

cbr = CatBoostClassifier(verbose=100,
                         task_type="GPU")

parameters = {
    'depth': [5,10,20,50],
    'learning_rate': [0.01, 0.02, 0.03],
    'iterations': [300, 500, 1000]
 }

rand_cbc = RandomizedSearchCV(cbr,
                              n_iter=20,
                              param_distributions= parameters,
                              scoring='f1',
                              n_jobs= -2,
                              cv=3,
                              random_state=42)

rand_cbc.fit(X_train, y_train.values)

best_rand_cbc = rand_cbc.best_score_

time_cbc = time.time() - start_time

print("Лучшие параметры для модели логистической регрессии с "\
    "использованием кросс-валидации:", rand_cbc.best_params_)
print("Наибольшее значение метрики F1 для модели логистической регрессии "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", rand_cbc)
print(f'Время выполнения - {round(time_cbc,2)}  сек.')

0:	learn: 0.5605250	total: 41.9ms	remaining: 2.05s
49:	learn: 0.0165284	total: 1.82s	remaining: 0us
0:	learn: 0.5674859	total: 28.6ms	remaining: 1.4s
49:	learn: 0.0184810	total: 1.48s	remaining: 0us
0:	learn: 0.5674322	total: 12.7ms	remaining: 622ms
49:	learn: 0.0167282	total: 1.17s	remaining: 0us
0:	learn: 0.6650552	total: 218ms	remaining: 1.96s
9:	learn: 0.4585873	total: 2.04s	remaining: 0us
0:	learn: 0.6668371	total: 115ms	remaining: 1.03s
9:	learn: 0.4647902	total: 1.35s	remaining: 0us
0:	learn: 0.6643910	total: 112ms	remaining: 1.01s
9:	learn: 0.4530894	total: 1.08s	remaining: 0us
0:	learn: 0.6648672	total: 17.3ms	remaining: 156ms
9:	learn: 0.4564276	total: 283ms	remaining: 0us
0:	learn: 0.6664840	total: 30.3ms	remaining: 273ms
9:	learn: 0.4681792	total: 301ms	remaining: 0us
0:	learn: 0.6664413	total: 17.5ms	remaining: 158ms
9:	learn: 0.4568692	total: 146ms	remaining: 0us
0:	learn: 0.5614388	total: 79.2ms	remaining: 23.7s
100:	learn: 0.0044513	total: 13.2s	remaining: 26s
200:	lear

In [ ]:
catbost_f1 = round(f1_score(y_val, rand_cbc.best_estimator_.predict(X_val)), 5)
catbost_f1

0.94737

In [ ]:
print(f'Значение метрики f1 = {catbost_f1}')

Значение метрики f1 = 0.9474


### LogisticRegression

In [ ]:
start_time = time.time()

log_reg = LogisticRegression(solver='liblinear',
                             random_state=42,
                             class_weight= 'balanced') #максимальное количество итераций 1000

parameters = {'max_iter': range (100, 1000, 100),
              'C': np.arange(0.1, 1.0, 0.1)} #перебор гиперпараметров

rand_log_reg = RandomizedSearchCV(log_reg,
                                  n_iter=20,
                                  param_distributions= parameters,
                                  scoring='f1',
                                  n_jobs= -2,
                                  cv=3,
                                  random_state=42)

rand_log_reg.fit(X_train, y_train.values)

best_log_reg = rand_log_reg.best_score_

time_lr = time.time() - start_time

print("Лучшие параметры для модели логистической регрессии с "\
    "использованием кросс-валидации:", rand_log_reg.best_params_)
print("Наибольшее значение метрики F1 для модели логистической регрессии "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_log_reg)
print(f'Время выполнения - {round(time_lr,2)}  сек.')

Лучшие параметры для модели логистической регрессии с использованием кросс-валидации: {'max_iter': 200, 'C': 0.9}
Наибольшее значение метрики F1 для модели логистической регрессии при лучших гиперпараметрах с использованием кросс-валидации: 0.9195867492898723
Время выполнения - 20.96  сек.


In [ ]:
log_reg_f1 = round(f1_score(y_val, rand_log_reg.best_estimator_.predict(X_val)), 5)
log_reg_f1

0.94964

### RandomForestClassifier

In [ ]:
start_time = time.time()

forest = RandomForestClassifier(class_weight= 'balanced',
                                random_state=42) #модель случайного леса

parameters = {'n_estimators': range (1, 300),
              'max_depth': range (1, 50)} #перебор гиперпараметров
#применение метода гридсёрч со встроенной кросс-валидацией к модели леса с перебором указанных параметров

randomized_forest = RandomizedSearchCV(forest,
                                       n_iter=20,
                                       param_distributions=parameters,
                                       scoring='f1',
                                       n_jobs= -2,
                                       cv=5,
                                       random_state=42)
#обучение модели
randomized_forest.fit(X_train, y_train.values)

#лучшее значение после перебора параметров
best_forest = randomized_forest.best_score_

time_forest = time.time() - start_time

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", randomized_forest.best_params_)
print("Наибольшее значение метрики F1 для модели случайного леса "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_forest)
print(f'Время выполнения - {round(time_forest,2)}')

Лучшие параметры для модели случайного леса с использованием кросс-валидации: {'n_estimators': 222, 'max_depth': 38}
Наибольшее значение метрики F1 для модели случайного леса при лучших гиперпараметрах с использованием кросс-валидации: 0.9303050833166069
Время выполнения - 289.38


In [ ]:
fandfor_f1 = round(f1_score(y_val, randomized_forest.best_estimator_.predict(X_val)), 5)
fandfor_f1

0.94737

In [ ]:
models = ['CatBoostClassifier', 'LogisticRegression', 'RandomForestClassifier']
F1 = [catbost_f1, log_reg_f1, fandfor_f1]
total_time = [round(time_cbc,2), round(time_lr,2), round(time_forest,2)]

result = pd.DataFrame({
    'Модель': models,
    'F1': F1,
    'Время выполнения': total_time
})
result.sort_values(by='F1')

,Модель,F1,Время выполнения
0,CatBoostClassifier,0.94737,5333.42
2,RandomForestClassifier,0.94737,289.38
1,LogisticRegression,0.94964,20.96


### Тест лучщей модели

In [ ]:
print(f'f1 модели catbost - {round(f1_score(y_test, rand_cbc.best_estimator_.predict(X_test)), 2)}')

f1 модели catbost - 0.93


## Выводы

Была использована модель BERT для создания признаков и обучена на этих лучшая модель молель CatBoost. Значение метрики F1 на тестовой выборке 0.93